In [1]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification

In [2]:
model = TFDistilBertForSequenceClassification.from_pretrained('./model')

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at ./model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [3]:
from transformers import TextClassificationPipeline, DistilBertTokenizerFast

In [4]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer)

In [24]:
test_fake = ['MULTIPLE FBI OFFICES Investigated Clinton Foundation BRIBERY Charges', '“financial crimes or influence-peddling”…DOJ Appeared To Want To Slow Down Or Stop Investigation","A stunning story of corruption, cronyism and cover-ups and oh yeah, the Clinton s FBI investigators from across the country have been following leads into reports of bribery involving the Clinton Foundation. Multiple field offices have been involved in the investigation. A report in Sunday s Wall Street Journal (WSJ) by Devlin Barrett revealed that agents assigned to the New York field office have been carrying the bulk of the work in investigating the Clinton Foundation. They have received assistance from the FBI field office in Little Rock according to  people familiar with the matter, the WSJ reported. Other offices, including Los Angeles and Washington, D.C., have been collecting evidence regarding  financial crimes or influence-peddling. As far back as February 2016, FBI agents made the presentation to the Department of Justice (DOJ), the WSJ s sources stated.  The meeting didn t go well,  they wrote. While some sources said the FBI s evidence was not strong enough, others believed the DOJ had no intention from the start of going any further. Barrett wrote that the DOJ officials were  stern, icy and dismissive of the case. Barrett wrote,  That was one of the weirdest meetings I ve ever been to,  one participant told others afterward, according to people familiar with the matter. It appears there was rift between the FBI and the DOJ about how to move forward with the investigation. Agents in the Washington office were directed to focus on a separate issue related to the actions of former Virginia Governor and Clinton Foundation Board member Terry McAuliffe. Agents inside the FBI believed they could build a stronger case if the investigation of McAuliffe and the foundation were combined.FBI Deputy Director Andrew McCabe seemed to be caught in the middle of the fight between DOJ officials who appeared to want to slow down or shut down the investigation and FBI agents who were eager to pour more resources into the investigation.Barrett wrote,  Are you telling me that I need to shut down a validly predicated investigation?  Mr. McCabe asked, according to people familiar with the conversation. After a pause, the official replied,  Of course not,  these people said. Some of the WSJ sources told Barrett that a  stand down  order had been given to the FBI agents by McCabe. Others denied that no such order was given.The story gets more complicated when you factor in that McCabe s wife, Dr. Jill McCabe had received a $467,500 campaign contribution in 2015 for a state senate race from McAuliffe.For entire story: Breitbart']

In [23]:
test_real = ['U.S. oil industry cheers Trump energy pick, seeks gas export boost', 'WASHINGTON (Reuters) - The U.S. oil and gas industry on Wednesday welcomed President-elect Donald Trump’s choice of former Texas Governor Rick Perry to head the U.S. Department of Energy, and wasted no time making its first specific request of him: to support increased exports of America’s natural gas overseas. Trump named Perry as his pick for the top U.S. energy job on Wednesday morning, handing the portfolio to a climate change skeptic with close ties to the oil and gas industry, and who previously proposed abolishing the department. The choice adds to a list of drilling proponents who have been tapped for top jobs in Trump’s administration, worrying environmental groups but fitting neatly with Trump’s promise to revive oil and gas drilling and coal mining as president by cutting back on federal regulation. Jack Gerard, president of the Washington-based American Petroleum Institute representing oil and natural gas companies, said he welcomed Perry’s nomination, and called on him to make increasing exports of U.S. natural gas a “top priority.” “As the former governor of Texas, Rick Perry knows the important impact that energy production has on our nation’s economy. In his new role at the Energy Department, he has the opportunity to encourage increased exports of domestically produced natural gas,” he said in a statement. Natural gas companies are eager to access foreign markets for their supply after a decade-long drilling boom that triggered a domestic glut and depressed prices.  The oil industry successfully lobbied for an end to a decades-old crude oil export ban in December 2015 following a slump in prices, a move meant to help American companies weather lower prices at home. There is no ban on natural gas exports, but U.S. law requires American companies to obtain authorization from the Energy Department before being able to ship it overseas, and there are tough permitting requirements for building the specialized facilities that make shipping gas possible. The United States exported its first cargo of liquefied natural gas earlier this year from an export facility on the Gulf Coast, but the industry has complained that boosting exports to match global demand has been constrained by a slow and opaque bureaucratic process. U.S. energy exports have long been a contentious political issue, dividing lawmakers seeking to balance the benefits of low consumer prices at home and American energy independence against opportunities for companies to expand access to potentially lucrative foreign markets. Trump, who takes office on Jan. 20, has made energy a central part of his agenda, vowing a revival in the oil, gas and coal industries, and picking industry allies for top administration jobs.  On Tuesday, Trump selected oil giant Exxon Mobil Corp’s CEO Rex Tillerson as secretary of state. His Environmental Protection Agency pick is Oklahoma Attorney General Scott Pruitt, a foe of Democratic President Barack Obama’s initiatives to combat climate change. Trump said during his campaign he would pull the United States out of a global deal to curb emissions of carbon dioxide signed in Paris last year, though he has since confused observers by saying he would keep an “open mind” on the accord. An overwhelming number of scientists say carbon dioxide from burning fossil fuels contributes to changes to the climate that are leading to sea level rise, droughts and more frequent violent storms. Trump’s transition team said Perry’s tenure leading Texas, the nation’s second most populous state and a major producer of oil, gas and wind power, from 2000 until 2015 made him a strong pick for energy secretary.  Trump said in a statement Perry had created “a business climate that produced millions of new jobs and lower energy prices in his state, and he will bring that same approach to our entire country as secretary of energy.” The department is responsible for U.S. energy policy and oversees the nation’s nuclear weapons program.  Perry is a former presidential rival and critic of Trump who unsuccessfully sought the Republican nomination in 2016 and 2012. His nomination requires Senate confirmation. If confirmed, he would replace Ernest Moniz, a physicist who was one of the chief negotiators in last year’s Iran nuclear deal. After his tenure as Texas governor, Perry joined the board of directors of Energy Transfer Partners, a Texas-based company building the Dakota Access Pipeline in North Dakota that has been stalled by protests by the Standing Rock Sioux tribe and supporters. Anthony Scaramucci, a member of the Trump transition team’s executive committee, said the president-elect was focused on preserving the environment as well as boosting U.S. energy production.  “Whether you believe in climate change or not, we want clean air, we want clean water for the American people. What we also want is energy independence,” Scaramucci told CNN. Billionaire environmental activist Tom Steyer, who backed Trump’s Democratic opponent Hillary Clinton in the Nov. 8 election, said he hoped Perry’s nomination would be blocked. “Trump no longer has to abolish departments. He can dismantle them from within,” Steyer said. “It’s now up to the Senate to defend our health, our economy and our democracy by defeating this nomination.”']

In [29]:
test_somefakebs = ['Coronavirus Bioweapon – How China Stole Coronavirus From Canada And Weaponized It', '''Last year a mysterious shipment was caught smuggling Coronavirus from Canada. It was traced to Chinese agents working at a Canadian lab. Subsequent investigation by GreatGameIndia linked the agents to Chinese Biological Warfare Program from where the virus is suspected to have leaked causing the Wuhan Coronavirus outbreak.

Note: BuzzFeed Reporter Who Attacked GreatGameIndia’s Coronavirus Bioweapon Story, Fired For Plagiarism

The findings of this investigation has been corroborated by none other than the Bioweapons expert Dr. Francis Boyle who drafted the Biological Weapons Convention Act followed by many nations. The report has caused a major international controversy and is suppressed actively by a section of mainstream media.

Infographic based on this report – The Secret History Of Coronavirus

''']

In [22]:
def run(input):
    tokens = tokenizer(*input, truncation=True, max_length=512, padding="max_length",return_tensors="tf")
    output = model(tokens)
    predictions = tf.nn.softmax(output[0], axis=-1)
    return ['real', 'fake'][tf.argmax(predictions, axis=1)[0].numpy()]

In [27]:
run(test_fake)

'fake'

In [26]:
run(test_real)

'real'

In [30]:
run(test_somefakebs)

'fake'

### start exporting model to one that can be loaded with tensorflow.js

In [3]:
callable = tf.function(model.call)

In [4]:
max_sequence_length = 512

In [5]:
concrete_function = callable.get_concrete_function([tf.TensorSpec([None, max_sequence_length], tf.int32, name="input_ids"), tf.TensorSpec([None, max_sequence_length], tf.int32, name="attention_mask")])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [6]:
tf.saved_model.save(model, 'distilbertu_ISOT', signatures=concrete_function)

INFO:tensorflow:Assets written to: distilbertu_ISOT\assets
INFO:tensorflow:Assets written to: distilbertu_ISOT\assets
